<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_4_1___sql_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
! pip install -qU cohere langchain langchain_cohere langchain_experimental

Import biblioteki "cohere" został wykonany przy użyciu polecenia pip w trybie cichym (-q). Flaga -U wymusza aktualizację do najnowszej wersji.

Instalowane są następujące biblioteki:
- cohere - główna biblioteka do obsługi modeli językowych Cohere
- langchain - framework do budowania aplikacji opartych o modele językowe  
- langchain_cohere - integracja LangChain z modelami Cohere
- langchain_experimental - eksperymentalne funkcje LangChain

Ten jeden wiersz kodu przygotowuje środowisko Pythona instalując wszystkie niezbędne zależności do pracy z modelami Cohere poprzez framework LangChain.

In [ ]:
from langchain_cohere import ChatCohere, create_sql_agent
from langchain_community.utilities.sql_database import SQLDatabase
import os
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
import os
from google.colab import userdata

Ten kod importuje niezbędne elementy do pracy z bazami danych SQL i modelami Cohere:

Z modułu langchain_cohere importowane są:
- ChatCohere - klasa do komunikacji z modelami czatowymi Cohere
- create_sql_agent - funkcja tworząca agenta do pracy z SQL

Z langchain_community.utilities.sql_database importowany jest:
- SQLDatabase - klasa do obsługi baz danych SQL

Z modułu langchain.agents pochodzi:
- AgentExecutor - wykonawca poleceń agenta

Z langchain_cohere.react_multi_hop.agent:
- create_cohere_react_agent - funkcja tworząca agenta zdolnego do wieloetapowego rozumowania

Z langchain_core.prompts:
- ChatPromptTemplate - szablon do formatowania promptów dla czata

Z langchain_community.agent_toolkits:
- SQLDatabaseToolkit - zestaw narzędzi do pracy z bazami SQL

Z google.colab zaimportowano:
- userdata - moduł do obsługi danych użytkownika w Google Colab

Dodatkowo zaimportowano moduł os do operacji na systemie operacyjnym.

In [ ]:
class CFG:
  model = "command-r-plus"
  db ='/content/Chinook.db'
  temperature = 0.1

In [ ]:
# api keys
os.environ['COHERE_API_KEY'] = userdata.get('cohereprod')

In [ ]:

llm = ChatCohere(model = CFG.model, temperature = CFG.temperature, verbose=True)
db = SQLDatabase.from_uri(f"sqlite:///{CFG.db}")

Ten fragment kodu tworzy dwa kluczowe obiekty:

1. llm - to instancja modelu językowego ChatCohere, gdzie:
   - model ustawiany jest zgodnie z wartością CFG.model
   - temperature (parametr losowości odpowiedzi) pobierany jest z CFG.temperature
   - verbose=True włącza szczegółowe logowanie działań modelu

2. db - to obiekt bazy danych SQLite, który:
   - tworzy połączenie z bazą danych SQLite
   - ścieżka do bazy jest konstruowana z CFG.db
   - używa formatu URI dla połączenia (sqlite:///)

Te dwa obiekty są niezbędne do pracy z bazą danych poprzez interfejs języka naturalnego - llm będzie interpretował zapytania użytkownika, a db umożliwi wykonywanie operacji na bazie danych.

# Prosty agent

In [ ]:
agent_executor = create_sql_agent(llm, db=db, verbose=True)

Ta linia tworzy wykonawcę agenta SQL, który łączy model językowy z bazą danych.

create_sql_agent tworzy specjalnego agenta, który:
- wykorzystuje wcześniej zdefiniowany model językowy (llm)
- ma dostęp do bazy danych (db)
- będzie działał w trybie szczegółowego logowania (verbose=True)

Agent ten potrafi:
- interpretować zapytania w języku naturalnym
- przekształcać je na zapytania SQL
- wykonywać te zapytania na bazie danych
- formułować odpowiedzi w języku naturalnym na podstawie otrzymanych wyników

Jest to kluczowy element łączący możliwości modelu językowego z operacjami bazodanowymi.

In [ ]:
agent_executor.invoke({
   "input": 'what tables are available?',
})



> Entering new Cohere SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: I will use the sql_db_list_tables tool to find out what tables are available in the database.

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe following tables are available in the database:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track

> Finished chain.


{'input': 'what tables are available?',
 'output': 'The following tables are available in the database:\n- Album\n- Artist\n- Customer\n- Employee\n- Genre\n- Invoice\n- InvoiceLine\n- MediaType\n- Playlist\n- PlaylistTrack\n- Track'}

To polecenie uruchamia agenta z prośbą o wyświetlenie dostępnych tabel w bazie danych.

Metoda invoke() wykonuje następujące działania:
- przyjmuje słownik z kluczem "input" zawierającym zapytanie w języku naturalnym
- agent przetwarza tekst "what tables are available?"
- następuje automatyczna konwersja tego zapytania na odpowiednie polecenie SQL (prawdopodobnie SHOW TABLES lub podobne)
- wyniki zapytania są pobierane z bazy danych
- agent formatuje te wyniki w czytelny sposób

Dzięki parametrowi verbose=True, który został wcześniej ustawiony, zobaczymy w konsoli cały proces rozumowania agenta oraz poszczególne kroki jego działania.

In [ ]:
agent_executor.invoke({
   "input": "who is the best customer? Please write and execute the correct sql query to answer this question, and give me the result!",
})



> Entering new Cohere SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: I will first list the tables in the database, then I will query the schema of the most relevant tables to answer the question.

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer,Invoice,InvoiceLine'}`
responded: The tables in the database are: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.

The most relevant tables to answer the question "who is the best customer?" are Customer, Invoice, and InvoiceLine. I will now query the schema of these tables.


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalC

{'input': 'who is the best customer? Please write and execute the correct sql query to answer this question, and give me the result!',
 'output': 'The most relevant tables to answer the question "who is the best customer?" are Customer, Invoice, and InvoiceLine. \n\nThe Customer table contains information about each customer, such as their ID, name, company, address, and contact details. The Invoice table contains information about each invoice, including the customer ID, invoice date, billing address, and total amount. The InvoiceLine table contains details about each item on an invoice, such as the track ID, unit price, and quantity.\n\nBased on this information, we can assume that the "best customer" is the one who has spent the most money. To find out who this is, we need to write a query that calculates the total amount spent by each customer.\n\nHere is the SQL query to answer the question:\n```sql\nSELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) AS TotalSpent\nFROM Cu

To polecenie każe agentowi znaleźć najlepszego klienta w bazie danych.

Podczas wykonania agent: analizuje pytanie o najlepszego klienta

Dzięki włączonemu trybowi verbose zobaczymy cały proces rozumowania i poszczególne kroki wykonania tego zadania.

In [ ]:
agent_executor.invoke({
   "input": "who is the best customer? The customer who has spent the most money is the best. Please write and execute the correct sql query to answer this question, and give me the result!",
})



> Entering new Cohere SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: I will first list the tables in the database, then I will query the schema of the most relevant tables to answer the question.

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`
responded: The tables in the database are: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.

The most relevant tables to answer the question are Customer, Invoice and InvoiceLine. I will now query the schema of these tables.


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone

{'input': 'who is the best customer? The customer who has spent the most money is the best. Please write and execute the correct sql query to answer this question, and give me the result!',
 'output': 'The best customer is the one with CustomerId 6, who has spent a total of $49.62.'}

To polecenie jest bardziej precyzyjne - każe znaleźć klienta, który wydał najwięcej pieniędzy.

Agent wykona następujące działania:
1. przeanalizuje strukturę bazy danych szukając tabel z:
   - informacjami o klientach
   - danymi o zakupach/transakcjach
   - kwotami zamówień

2. stworzy zapytanie SQL, które:
   - połączy odpowiednie tabele  
   - zsumuje wszystkie wydatki dla każdego klienta
   - posortuje wyniki malejąco według sumy wydatków
   - wybierze klienta z najwyższą sumą

3. wykona zapytanie na bazie danych

4. zwróci wynik zawierający:
   - dane identyfikacyjne klienta (np. imię, nazwisko, ID)
   - całkowitą kwotę jego wydatków
   - możliwe dodatkowe informacje potwierdzające wynik

W logach zobaczymy dokładne zapytanie SQL i proces dochodzenia do wyniku.

# Reaktywny agent

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

print('**List of pre-defined Langchain Tools**')
print([tool.name for tool in tools])
print('')
print('**Context to pass to LLM on tables**')
print(context)

**List of pre-defined Langchain Tools**
['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

**Context to pass to LLM on tables**
{'table_info': '\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(

Ten fragment kodu tworzy i analizuje zestaw narzędzi do pracy z bazą danych.

Najpierw tworzony jest toolkit - zestaw narzędzi SQL, który łączy:
- bazę danych (db)
- model językowy (llm)

Następnie pobierane są:
- context - informacje o strukturze bazy danych
- tools - lista dostępnych narzędzi

Kod wyświetla dwie informacje:
1. Listę predefiniowanych narzędzi LangChain - są to nazwy wszystkich dostępnych narzędzi do pracy z bazą
2. Kontekst zawierający szczegółowe informacje o:
   - strukturze tabel
   - kolumnach
   - typach danych
   - relacjach między tabelami

Te informacje są niezbędne dla modelu językowego do prawidłowego formułowania i wykonywania zapytań SQL.

In [ ]:
# define the prompt template
prompt = ChatPromptTemplate.from_template("{input}")

# instantiate the ReAct agent
agent = create_cohere_react_agent(
   llm=llm,
   tools=tools,
   prompt=prompt,
)
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               return_intermediate_steps=True
                    )

Ten kod konfiguruje nowego, bardziej zaawansowanego agenta typu ReAct.

Najpierw tworzony jest prosty szablon promptu, który będzie przekazywał zapytania użytkownika bez modyfikacji.

Następnie tworzony jest agent ReAct:
- wykorzystuje wcześniej zdefiniowany model (llm)
- ma dostęp do utworzonych narzędzi (tools)
- używa zdefiniowanego szablonu promptu

Agent jest następnie umieszczany w wykonawcy (AgentExecutor) z parametrami:
- agent - utworzony agent ReAct
- tools - zestaw dostępnych narzędzi
- verbose=True - włącza szczegółowe logowanie
- return_intermediate_steps=True - zwraca wszystkie pośrednie kroki rozumowania

Ten agent jest bardziej zaawansowany, ponieważ:
- potrafi rozbijać złożone zadania na mniejsze kroki
- wykorzystuje poprzednie wyniki do planowania kolejnych działań
- pokazuje proces swojego rozumowania

In [ ]:
agent_executor.invoke({
   "input": 'what tables are available?',
})



> Entering new AgentExecutor chain...

I will use the sql_db_list_tables tool to find out what tables are available.
{'tool_name': 'sql_db_list_tables', 'parameters': {'tool_input': ''}}
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackRelevant Documents: 0
Cited Documents: 0
Answer: The following tables are available: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.
Grounded answer: The following tables are available: <co: 0>Album</co: 0>, <co: 0>Artist</co: 0>, <co: 0>Customer</co: 0>, <co: 0>Employee</co: 0>, <co: 0>Genre</co: 0>, <co: 0>Invoice</co: 0>, <co: 0>InvoiceLine</co: 0>, <co: 0>MediaType</co: 0>, <co: 0>Playlist</co: 0>, <co: 0>PlaylistTrack</co: 0>, <co: 0>Track</co: 0>.

> Finished chain.


{'input': 'what tables are available?',
 'output': 'The following tables are available: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.',
 'citations': [CohereCitation(start=36, end=41, text='Album', documents=[{'output': 'Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', 'id': 'doc_0'}], document_ids={'doc_0'}),
  CohereCitation(start=43, end=49, text='Artist', documents=[{'output': 'Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', 'id': 'doc_0'}], document_ids={'doc_0'}),
  CohereCitation(start=51, end=59, text='Customer', documents=[{'output': 'Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', 'id': 'doc_0'}], document_ids={'doc_0'}),
  CohereCitation(start=61, end=69, text='Employee', documents=[{'output': 'Album, Artist, Customer, Employee, Genre, Inv

In [ ]:
agent_executor.invoke({
   "input": 'show the first row of the Playlist and Genre tables?',
})




> Entering new AgentExecutor chain...

I will use the sql_db_query tool to find the first row of the Playlist and Genre tables.
{'tool_name': 'sql_db_query', 'parameters': {'query': 'SELECT * FROM Playlist LIMIT 1;\nSELECT * FROM Genre LIMIT 1;'}}


Warning: You can only execute one statement at a time.

In [ ]:
agent_executor.invoke({
   "input": 'which countries have the most invoices?',
})



> Entering new AgentExecutor chain...

I will run a SQL query to find the countries with the most invoices.
{'tool_name': 'sql_db_query', 'parameters': {'query': 'SELECT country, COUNT(*) AS invoice_count FROM invoices GROUP BY country ORDER BY invoice_count DESC LIMIT 10'}}
Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT country, COUNT(*) AS invoice_count FROM invoices GROUP BY country ORDER BY invoice_count DESC LIMIT 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
The query failed because the table 'invoices' does not exist. I will run a query to list all the tables in the database and then use this information to rewrite the query.
{'tool_name': 'sql_db_list_tables', 'parameters': {'tool_input': ''}}
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
The table name is 'Invoice' rather than 'invoices'. I will rewrite the query and run it again.
{'tool_name': 'sql_db_query', 'parameters':

{'input': 'which countries have the most invoices?',
 'output': 'The countries with the most invoices are:\n1. USA (91)\n2. Canada (56)\n3. France (35)\n4. Brazil (35)\n5. Germany (28)\n6. United Kingdom (21)\n7. Portugal (14)\n8. Czech Republic (14)\n9. India (13)\n10. Sweden (7)',
 'citations': [CohereCitation(start=45, end=52, text='USA (91', documents=[{'output': "[('USA', 91), ('Canada', 56), ('France', 35), ('Brazil', 35), ('Germany', 28), ('United Kingdom', 21), ('Portugal', 14), ('Czech Republic', 14), ('India', 13), ('Sweden', 7)]", 'id': 'doc_4'}], document_ids={'doc_4'}),
  CohereCitation(start=57, end=67, text='Canada (56', documents=[{'output': "[('USA', 91), ('Canada', 56), ('France', 35), ('Brazil', 35), ('Germany', 28), ('United Kingdom', 21), ('Portugal', 14), ('Czech Republic', 14), ('India', 13), ('Sweden', 7)]", 'id': 'doc_4'}], document_ids={'doc_4'}),
  CohereCitation(start=72, end=82, text='France (35', documents=[{'output': "[('USA', 91), ('Canada', 56), ('Franc

In [ ]:
output = agent_executor.invoke({
   "input": 'who is the best customer? The customer who has spent the most money is the best.',
})



> Entering new AgentExecutor chain...

I will write a SQL query to find the customer who has spent the most money.
{'tool_name': 'sql_db_query', 'parameters': {'query': 'SELECT customer_name, SUM(total_cost) AS total_spent FROM orders GROUP BY customer_name ORDER BY total_spent DESC LIMIT 1;'}}
Error: (sqlite3.OperationalError) no such table: orders
[SQL: SELECT customer_name, SUM(total_cost) AS total_spent FROM orders GROUP BY customer_name ORDER BY total_spent DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
I received an error message saying that there is no such table as 'orders'. I will use the sql_db_list_tables tool to find out what tables are available.
{'tool_name': 'sql_db_list_tables', 'parameters': {'tool_input': ''}}
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
I found that there is a 'Customer' table. I will use the sql_db_schema tool to find out what columns are in the 'Customer' ta

In [ ]:
print(output['output'])

The best customer is Helena Holý, who has spent a total of 49.62.


# Reakcja z kontekstem

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()

In [ ]:
print('Table Names')
print(context['table_names'])
print('Table Schemas')
print(context['table_info'])

Table Names
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Table Schemas

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"

In [ ]:
preamble="""## Task And Context
You use your advanced complex reasoning capabilities to help people by answering their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You may need to use multiple tools in parallel or sequentially to complete your task. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.

## Additional Information
You are an expert who answers the user's question by creating SQL queries and executing them.
You are equipped with a number of relevant SQL tools.

Here is information about the database:
{schema_info}
""".format(schema_info=context)

In [ ]:
output = agent_executor.invoke({
   "input": 'provide the name of the best customer? The customer who has spent the most money is the best.',
   "preamble": preamble
})



> Entering new AgentExecutor chain...

I will first write a query to find the customer who has spent the most money. Then, I will write another query to find the name of that customer.
{'tool_name': 'sql_db_query', 'parameters': {'query': 'SELECT CustomerId, SUM(Total) AS TotalSpent FROM Invoice GROUP BY CustomerId ORDER BY TotalSpent DESC LIMIT 1'}}
[(6, 49.620000000000005)]
I found that the customer with ID 6 has spent the most money. Now, I will write another query to find the name of the customer with ID 6.
{'tool_name': 'sql_db_query', 'parameters': {'query': 'SELECT FirstName, LastName FROM Customer WHERE CustomerId = 6'}}
[('Helena', 'Holý')]Relevant Documents: 0,1
Cited Documents: 0,1
Answer: The best customer is Helena Holý. She has spent a total of $49.62.
Grounded answer: The best customer is <co: 1>Helena Holý</co: 1>. She has spent a total of <co: 0>$49.62</co: 0>.

> Finished chain.


In [ ]:
print(output['output'])